<a href="https://colab.research.google.com/github/smurthy5/DeepLabCut/blob/main/notebooks/DLC_Octopus_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepLabCut for your standard (single animal) projects!

Some useful links:

- [DeepLabCut's GitHub: github.com/DeepLabCut/DeepLabCut](https://github.com/DeepLabCut/DeepLabCut)
- [DeepLabCut's Documentation: User Guide for Single Animal projects](https://deeplabcut.github.io/DeepLabCut/docs/standardDeepLabCut_UserGuide.html)


This notebook illustrates how to use the cloud to:
- create a training set
- train a network
- evaluate a network
- create simple quality check plots
- analyze novel videos!

### This notebook assumes you already have a project folder with labeled data!

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.

This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

Nath\*, Mathis\* et al.: Using DeepLabCut for markerless pose estimation during behavior across species. Nature Protocols, 2019.


Paper: https://www.nature.com/articles/s41596-019-0176-0

Pre-print: https://www.biorxiv.org/content/biorxiv/early/2018/11/24/476531.full.pdf


## First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"

As the COLAB environments were updated to CUDA 12.X and Python 3.11, we need to install DeepLabCut and TensorFlow in a distinct way to get TensorFlow to connect to the GPU.

In [3]:
# this will take a couple of minutes to install all the dependencies!
!pip install --pre deeplabcut

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python-headless to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.0/948.0 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.6/118.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.6 MB/s eta 0:00:00
   ━

**(Be sure to click "RESTART RUNTIME" if it is displayed above before moving on !)** You will see this button at the output of the cells above ^.

In [1]:
import deeplabcut

Loading DLC 3.0.0rc10...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


## Link your Google Drive (with your labeled data, or the demo data):

### First, place your project folder into you google drive! "i.e. move the folder named "Project-YourName-TheDate" into google drive.

In [6]:
# Now, let's link to your GoogleDrive. Run this cell and follow the authorization instructions:
# (We recommend putting a copy of the github repo in your google drive if you are using the demo "examples")

from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


YOU WILL NEED TO EDIT THE PROJECT PATH **in the config.yaml file** TO BE SET TO YOUR GOOGLE DRIVE LINK!

Typically, this will be: `/content/drive/My Drive/yourProjectFolderName`


In [7]:
# PLEASE EDIT THIS:
project_folder_name = "OctopusArmTracking-Sonia-2025-07-01"
video_type = "avi" #, mp4, MOV, or avi, whatever you uploaded!

# No need to edit this, we are going to assume you put videos you want to analyze
# in the "videos" folder, but if this is NOT true, edit below:
videofile_path = [f"/content/drive/My Drive/OctopusArmTracking-Sonia-2025-07-01/videos/"]
print(videofile_path)

# The prediction files and labeled videos will be saved in this `labeled-videos` folder
# in your project folder; if you want them elsewhere, you can edit this;
# if you want the output files in the same folder as the videos, set this to an empty string.
destfolder = f"/content/drive/My Drive/OctopusArmTracking-Sonia-2025-07-01/labeled-videos"

#No need to edit this, as you set it when you passed the ProjectFolderName (above):
path_config_file = f"/content/drive/My Drive/OctopusArmTracking-Sonia-2025-07-01/config.yaml"
print(path_config_file)

# This creates a path variable that links to your Google Drive project

['/content/drive/My Drive/OctopusArmTracking-Sonia-2025-07-01/videos/']
/content/drive/My Drive/OctopusArmTracking-Sonia-2025-07-01/config.yaml


In [12]:
import os

ld_path = os.path.join(project_path, "labeled-data")

for folder in os.listdir(ld_path):
    folder_path = os.path.join(ld_path, folder)
    if not os.path.isdir(folder_path):
        continue

    h5_path = os.path.join(folder_path, "CollectedData_Sonia.h5")
    if os.path.exists(h5_path):
        print(f"✅ Annotated: {folder}")
    else:
        print(f"❌ Not annotated (or bad): {folder}")


✅ Annotated: arm1_chenodeoxycholicAc-03262025155127-0000
❌ Not annotated (or bad): arm5_progesterone-03252025190202-0000_labeled
✅ Annotated: 200microL_100microM_chloroquine_L2-01102025124342-0000 - Copy
✅ Annotated: arm4_chloroquine-03142025153729-0000
✅ Annotated: arm5_progesterone-03252025190202-0000
❌ Not annotated (or bad): R2_Amylacetate-06112024151423-0000_labeled
❌ Not annotated (or bad): T4_chenodexoy-02112025163907-0000_labeled
❌ Not annotated (or bad): R2_nootK-05232024143631-0000_labeled
❌ Not annotated (or bad): R4_chloroq-05232024145138-0000_labeled
❌ Not annotated (or bad): R3_pregenolone-06112024145204-0000_labeled
❌ Not annotated (or bad): L2_octopamine-06132024145544-0000_labeled
❌ Not annotated (or bad): OR270_arm5_chenodeoxycholicAc_01-02192025200046-0000_labeled
❌ Not annotated (or bad): Hect_NootK-05232024141434-0000_labeled
✅ Annotated: R2_Amylacetate-06112024151423-0000
❌ Not annotated (or bad): OR270_arm5_estradiol_01-02192025200626-0000_labeled
✅ Annotated: R3

In [13]:
import shutil
import os

train_dataset_path = os.path.join(project_path, "training-datasets")

if os.path.exists(train_dataset_path):
    print("Deleting old training-datasets folder...")
    shutil.rmtree(train_dataset_path)
else:
    print("No existing training-datasets folder found.")


Deleting old training-datasets folder...


In [14]:
import os

ld_path = os.path.join(project_path, "labeled-data")

for folder in os.listdir(ld_path):
    folder_path = os.path.join(ld_path, folder)
    if not os.path.isdir(folder_path):
        continue

    h5_path = os.path.join(folder_path, "CollectedData_Sonia.h5")
    if os.path.exists(h5_path):
        print(f"✅ Annotated: {folder}")
    else:
        print(f"❌ Not annotated (or bad): {folder}")


✅ Annotated: arm1_chenodeoxycholicAc-03262025155127-0000
✅ Annotated: 200microL_100microM_chloroquine_L2-01102025124342-0000 - Copy
✅ Annotated: arm4_chloroquine-03142025153729-0000
✅ Annotated: arm5_progesterone-03252025190202-0000
✅ Annotated: R2_Amylacetate-06112024151423-0000
✅ Annotated: R3_pregenolone-06112024145204-0000
✅ Annotated: L2_octopamine-06132024145544-0000
✅ Annotated: T4_chenodexoy-02112025163907-0000
✅ Annotated: R2_nootK-05232024143631-0000
✅ Annotated: arm1_chloroquine-03142025145416-0000
✅ Annotated: arm4_nootk_01-02172025175849-0000
✅ Annotated: R4_chloroq-05232024145138-0000
✅ Annotated: OR270_arm5_chenodeoxycholicAc_01-02192025200046-0000
✅ Annotated: 200microL_100microM_progesterone_hect -01102025121221-0000 - Copy
✅ Annotated: A1_harmane-06102024144751-0000
✅ Annotated: Hect_NootK-05232024141434-0000
✅ Annotated: OR270_arm5_estradiol_01-02192025200626-0000
✅ Annotated: A1_Glutamine-06102024143755-0000
✅ Annotated: AL1_progest-05072024144938-0000
✅ Annotated: 

## Create a training dataset:

### You must do this step inside of Colab

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models-pytorch** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pytorch_config.yaml**.

Now it is the time to start training the network!

In [8]:
# There are many more functions you can set here, including which network to use!
# Check the docstring for `create_training_dataset` for all options you can use!

deeplabcut.create_training_dataset(
  path_config_file, net_type="resnet_50", engine=deeplabcut.Engine.PYTORCH
)

/content/drive/My Drive/OctopusArmTracking-Sonia-2025-07-01/labeled-data/._arm1_chenodeoxycholicAc-03262025155127-0000/CollectedData_Sonia.h5  not found (perhaps not annotated).
/content/drive/My Drive/OctopusArmTracking-Sonia-2025-07-01/labeled-data/._arm4_chloroquine-03142025153729-0000/CollectedData_Sonia.h5  not found (perhaps not annotated).
/content/drive/My Drive/OctopusArmTracking-Sonia-2025-07-01/labeled-data/OR270_arm2_nootkatone_01-02192025182208-0000 10.40.33 AM/CollectedData_Sonia.h5  not found (perhaps not annotated).


KeyboardInterrupt: 

In [9]:
project_path = "/content/drive/MyDrive/OctopusArmTracking-Sonia-2025-07-01"


In [ ]:
import os

# Check if the training dataset folder is being created
train_dir = os.path.join(project_path, 'training-datasets')
os.listdir(train_dir) if os.path.exists(train_dir) else "Still working..."


In [9]:
import os

# Rename the long timestamp folder
bad_path = "/content/drive/MyDrive/OctopusArmTracking-Sonia-2025-07-01/labeled-data/OR270_arm2_nootkatone_01-02192025182208-0000 10.40.33 AM"
fixed_path = "/content/drive/MyDrive/OctopusArmTracking-Sonia-2025-07-01/labeled-data/OR270_arm2_nootkatone"

if os.path.exists(bad_path):
    os.rename(bad_path, fixed_path)
    print("Renamed successfully.")
else:
    print("Path not found.")


Path not found.


## Start training:
This function trains the network for a specific shuffle of the training dataset.

In [ ]:
# Let's also change the display and save_epochs just in case Colab takes away
# the GPU... If that happens, you can reload from a saved point using the
# `snapshot_path` argument to `deeplabcut.train_network`:
#   deeplabcut.train_network(..., snapshot_path="/content/.../snapshot-050.pt")

# Typically, you want to train to ~200 epochs. We set the batch size to 8 to
# utilize the GPU's capabilities.

# More info and there are more things you can set:
#   https://deeplabcut.github.io/DeepLabCut/docs/standardDeepLabCut_UserGuide.html#g-train-the-network

deeplabcut.train_network(
    path_config_file,
    shuffle=1,
    save_epochs=5,
    epochs=200,
    batch_size=8,
)

# This will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end.

Training with configuration:
data:
  bbox_margin: 20
  colormode: RGB
  inference:
    normalize_images: True
  train:
    affine:
      p: 0.5
      rotation: 30
      scaling: [0.5, 1.25]
      translation: 0
    crop_sampling:
      width: 448
      height: 448
      max_shift: 0.1
      method: hybrid
    gaussian_noise: 12.75
    motion_blur: True
    normalize_images: True
device: auto
metadata:
  project_path: /content/drive/My Drive/OctopusArmTracking-Sonia-2025-07-01
  pose_config_path: /content/drive/My Drive/OctopusArmTracking-Sonia-2025-07-01/dlc-models-pytorch/iteration-20000/OctopusArmTrackingJul1-trainset95shuffle1/train/pytorch_config.yaml
  bodyparts: ['base', 'arm1', 'arm2', 'mid', 'arm3', 'arm4', 'tip']
  unique_bodyparts: []
  individuals: ['animal']
  with_identity: None
method: bu
model:
  backbone:
    type: ResNet
    model_name: resnet50_gn
    output_stride: 16
    freeze_bn_stats: False
    freeze_bn_weights: False
  backbone_output_channels: 2048
  heads:
  

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

[timm/resnet50_gn.a1h_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
Data Transforms:
  Training:   Compose([
  Affine(always_apply=False, p=0.5, interpolation=1, mask_interpolation=0, cval=0, mode=0, scale={'x': (0.5, 1.25), 'y': (0.5, 1.25)}, translate_percent=None, translate_px={'x': (0, 0), 'y': (0, 0)}, rotate=(-30, 30), fit_output=False, shear={'x': (0.0, 0.0), 'y': (0.0, 0.0)}, cval_mask=0, keep_ratio=True, rotate_method='largest_box'),
  PadIfNeeded(always_apply=True, p=1.0, min_height=448, min_width=448, pad_height_divisor=None, pad_width_divisor=None, position=PositionType.CENTER, border_mode=0, value=None, mask_value=None),
  KeypointAwareCrop(always_apply=True, p=1.0, width=448, height=448, max_shift=0.1, crop_sampling='hybrid'),
  MotionBlur(always_apply=False, p=0.5, blur_limit=(3, 7), allow_shifted=True),
  GaussNoise(always_apply=False, p=0.5, var_limit=(0, 162.5625), per_channel=True, mean=0),
  Nor

Note, that **when you hit "STOP" you will get a `KeyboardInterrupt` "error"! No worries! :)**

## Start evaluating:
This function evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results-pytorch**

In [ ]:
deeplabcut.evaluate_network(path_config_file, plotting=True)

# Here you want to see a low pixel error! Of course, it can only be as
# good as the labeler, so be sure your labels are good!


## There is an optional refinement step you can do outside of Colab:
- if your pixel errors are not low enough, please check out the protocol guide on how to refine your network!
- You will need to adjust the labels **outside of Colab!** We recommend coming back to train and analyze videos...
- Please see the repo and protocol instructions on how to refine your data!

## Start Analyzing videos:
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides.

In [ ]:
deeplabcut.analyze_videos(
    path_config_file,
    videofile_path,
    videotype=video_type,
    destfolder=destfolder,
)

## Plot the trajectories of the analyzed videos:
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
deeplabcut.plot_trajectories(
    path_config_file,
    videofile_path,
    videotype=video_type,
    destfolder=destfolder,
)

Now you can look at the plot-poses file and check the "plot-likelihood.png" might want to change the "p-cutoff" in the config.yaml file so that you have only high confidnece points plotted in the video. i.e. ~0.8 or 0.9. The current default is 0.4.

## Create labeled video:
This function is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides.

In [ ]:
deeplabcut.create_labeled_video(
    path_config_file,
    videofile_path,
    videotype=video_type,
    destfolder=destfolder,
)